In [ ]:
# default_exp models.MINIROCKET

# MINIROCKET

> A Very Fast (Almost) Deterministic Transform for Time Series Classification.

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.external import *
from tsai.models.layers import *

In [ ]:
# hide

# # minirocket_multivariate.py from https://github.com/angus924/minirocket

# # Angus Dempster, Daniel F Schmidt, Geoffrey I Webb

# # MiniRocket: A Very Fast (Almost) Deterministic Transform for Time Series
# # Classification

# # https://arxiv.org/abs/2012.08791
# # Original code: https://github.com/angus924/minirocket

# from numba import njit, prange, vectorize
# import numpy as np


# @njit("float32[:](float32[:,:,:],int32[:],int32[:],int32[:],int32[:],float32[:])", fastmath = True, parallel = False, cache = True)
# def _fit_biases(X, num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, quantiles):

#     num_examples, num_channels, input_length = X.shape

#     # equivalent to:
#     # >>> from itertools import combinations
#     # >>> indices = np.array([_ for _ in combinations(np.arange(9), 3)], dtype = np.int32)
#     indices = np.array((
#         0,1,2,0,1,3,0,1,4,0,1,5,0,1,6,0,1,7,0,1,8,
#         0,2,3,0,2,4,0,2,5,0,2,6,0,2,7,0,2,8,0,3,4,
#         0,3,5,0,3,6,0,3,7,0,3,8,0,4,5,0,4,6,0,4,7,
#         0,4,8,0,5,6,0,5,7,0,5,8,0,6,7,0,6,8,0,7,8,
#         1,2,3,1,2,4,1,2,5,1,2,6,1,2,7,1,2,8,1,3,4,
#         1,3,5,1,3,6,1,3,7,1,3,8,1,4,5,1,4,6,1,4,7,
#         1,4,8,1,5,6,1,5,7,1,5,8,1,6,7,1,6,8,1,7,8,
#         2,3,4,2,3,5,2,3,6,2,3,7,2,3,8,2,4,5,2,4,6,
#         2,4,7,2,4,8,2,5,6,2,5,7,2,5,8,2,6,7,2,6,8,
#         2,7,8,3,4,5,3,4,6,3,4,7,3,4,8,3,5,6,3,5,7,
#         3,5,8,3,6,7,3,6,8,3,7,8,4,5,6,4,5,7,4,5,8,
#         4,6,7,4,6,8,4,7,8,5,6,7,5,6,8,5,7,8,6,7,8
#     ), dtype = np.int32).reshape(84, 3)

#     num_kernels = len(indices)
#     num_dilations = len(dilations)

#     num_features = num_kernels * np.sum(num_features_per_dilation)

#     biases = np.zeros(num_features, dtype = np.float32)

#     feature_index_start = 0

#     combination_index = 0
#     num_channels_start = 0

#     for dilation_index in range(num_dilations):

#         dilation = dilations[dilation_index]
#         padding = ((9 - 1) * dilation) // 2

#         num_features_this_dilation = num_features_per_dilation[dilation_index]

#         for kernel_index in range(num_kernels):

#             feature_index_end = feature_index_start + num_features_this_dilation

#             num_channels_this_combination = num_channels_per_combination[combination_index]

#             num_channels_end = num_channels_start + num_channels_this_combination

#             channels_this_combination = channel_indices[num_channels_start:num_channels_end]

#             _X = X[np.random.randint(num_examples)][channels_this_combination]

#             A = -_X          # A = alpha * X = -X
#             G = _X + _X + _X # G = gamma * X = 3X

#             C_alpha = np.zeros((num_channels_this_combination, input_length), dtype = np.float32)
#             C_alpha[:] = A

#             C_gamma = np.zeros((9, num_channels_this_combination, input_length), dtype = np.float32)
#             C_gamma[9 // 2] = G

#             start = dilation
#             end = input_length - padding

#             for gamma_index in range(9 // 2):

#                 C_alpha[:, -end:] = C_alpha[:, -end:] + A[:, :end]
#                 C_gamma[gamma_index, :, -end:] = G[:, :end]

#                 end += dilation

#             for gamma_index in range(9 // 2 + 1, 9):

#                 C_alpha[:, :-start] = C_alpha[:, :-start] + A[:, start:]
#                 C_gamma[gamma_index, :, :-start] = G[:, start:]

#                 start += dilation

#             index_0, index_1, index_2 = indices[kernel_index]

#             C = C_alpha + C_gamma[index_0] + C_gamma[index_1] + C_gamma[index_2]
#             C = np.sum(C, axis = 0)

#             biases[feature_index_start:feature_index_end] = np.quantile(C, quantiles[feature_index_start:feature_index_end])

#             feature_index_start = feature_index_end

#             combination_index += 1
#             num_channels_start = num_channels_end

#     return biases

# def _fit_dilations(input_length, num_features, max_dilations_per_kernel):

#     num_kernels = 84

#     num_features_per_kernel = num_features // num_kernels
#     true_max_dilations_per_kernel = min(num_features_per_kernel, max_dilations_per_kernel)
#     multiplier = num_features_per_kernel / true_max_dilations_per_kernel

#     max_exponent = np.log2((input_length - 1) / (9 - 1))
#     dilations, num_features_per_dilation = \
#     np.unique(np.logspace(0, max_exponent, true_max_dilations_per_kernel, base = 2).astype(np.int32), return_counts = True)
#     num_features_per_dilation = (num_features_per_dilation * multiplier).astype(np.int32) # this is a vector

#     remainder = num_features_per_kernel - np.sum(num_features_per_dilation)
#     i = 0
#     while remainder > 0:
#         num_features_per_dilation[i] += 1
#         remainder -= 1
#         i = (i + 1) % len(num_features_per_dilation)

#     return dilations, num_features_per_dilation

# # low-discrepancy sequence to assign quantiles to kernel/dilation combinations
# def _quantiles(n):
#     return np.array([(_ * ((np.sqrt(5) + 1) / 2)) % 1 for _ in range(1, n + 1)], dtype = np.float32)

# def fit(X, num_features = 10_000, max_dilations_per_kernel = 32):

#     _, num_channels, input_length = X.shape

#     num_kernels = 84

#     dilations, num_features_per_dilation = _fit_dilations(input_length, num_features, max_dilations_per_kernel)

#     num_features_per_kernel = np.sum(num_features_per_dilation)

#     quantiles = _quantiles(num_kernels * num_features_per_kernel)

#     num_dilations = len(dilations)
#     num_combinations = num_kernels * num_dilations

#     max_num_channels = min(num_channels, 9)
#     max_exponent = np.log2(max_num_channels + 1)

#     num_channels_per_combination = (2 ** np.random.uniform(0, max_exponent, num_combinations)).astype(np.int32)

#     channel_indices = np.zeros(num_channels_per_combination.sum(), dtype = np.int32)

#     num_channels_start = 0
#     for combination_index in range(num_combinations):
#         num_channels_this_combination = num_channels_per_combination[combination_index]
#         num_channels_end = num_channels_start + num_channels_this_combination
#         channel_indices[num_channels_start:num_channels_end] = np.random.choice(num_channels, num_channels_this_combination, replace = False)

#         num_channels_start = num_channels_end

#     biases = _fit_biases(X, num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, quantiles)

#     return num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, biases

# # _PPV(C, b).mean() returns PPV for vector C (convolution output) and scalar b (bias)
# @vectorize("float32(float32,float32)", nopython = True, cache = True)
# def _PPV(a, b):
#     if a > b:
#         return 1
#     else:
#         return 0

# @njit("float32[:,:](float32[:,:,:],Tuple((int32[:],int32[:],int32[:],int32[:],float32[:])))", fastmath = True, parallel = True, cache = True)
# def transform(X, parameters):

#     num_examples, num_channels, input_length = X.shape

#     num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, biases = parameters

#     # equivalent to:
#     # >>> from itertools import combinations
#     # >>> indices = np.array([_ for _ in combinations(np.arange(9), 3)], dtype = np.int32)
#     indices = np.array((
#         0,1,2,0,1,3,0,1,4,0,1,5,0,1,6,0,1,7,0,1,8,
#         0,2,3,0,2,4,0,2,5,0,2,6,0,2,7,0,2,8,0,3,4,
#         0,3,5,0,3,6,0,3,7,0,3,8,0,4,5,0,4,6,0,4,7,
#         0,4,8,0,5,6,0,5,7,0,5,8,0,6,7,0,6,8,0,7,8,
#         1,2,3,1,2,4,1,2,5,1,2,6,1,2,7,1,2,8,1,3,4,
#         1,3,5,1,3,6,1,3,7,1,3,8,1,4,5,1,4,6,1,4,7,
#         1,4,8,1,5,6,1,5,7,1,5,8,1,6,7,1,6,8,1,7,8,
#         2,3,4,2,3,5,2,3,6,2,3,7,2,3,8,2,4,5,2,4,6,
#         2,4,7,2,4,8,2,5,6,2,5,7,2,5,8,2,6,7,2,6,8,
#         2,7,8,3,4,5,3,4,6,3,4,7,3,4,8,3,5,6,3,5,7,
#         3,5,8,3,6,7,3,6,8,3,7,8,4,5,6,4,5,7,4,5,8,
#         4,6,7,4,6,8,4,7,8,5,6,7,5,6,8,5,7,8,6,7,8
#     ), dtype = np.int32).reshape(84, 3)

#     num_kernels = len(indices)
#     num_dilations = len(dilations)

#     num_features = num_kernels * np.sum(num_features_per_dilation)

#     features = np.zeros((num_examples, num_features), dtype = np.float32)

#     for example_index in prange(num_examples):

#         _X = X[example_index]

#         A = -_X          # A = alpha * X = -X
#         G = _X + _X + _X # G = gamma * X = 3X

#         feature_index_start = 0

#         combination_index = 0
#         num_channels_start = 0

#         for dilation_index in range(num_dilations):

#             _padding0 = dilation_index % 2

#             dilation = dilations[dilation_index]
#             padding = ((9 - 1) * dilation) // 2

#             num_features_this_dilation = num_features_per_dilation[dilation_index]

#             C_alpha = np.zeros((num_channels, input_length), dtype = np.float32)
#             C_alpha[:] = A

#             C_gamma = np.zeros((9, num_channels, input_length), dtype = np.float32)
#             C_gamma[9 // 2] = G

#             start = dilation
#             end = input_length - padding

#             for gamma_index in range(9 // 2):

#                 C_alpha[:, -end:] = C_alpha[:, -end:] + A[:, :end]
#                 C_gamma[gamma_index, :, -end:] = G[:, :end]

#                 end += dilation

#             for gamma_index in range(9 // 2 + 1, 9):

#                 C_alpha[:, :-start] = C_alpha[:, :-start] + A[:, start:]
#                 C_gamma[gamma_index, :, :-start] = G[:, start:]

#                 start += dilation

#             for kernel_index in range(num_kernels):

#                 feature_index_end = feature_index_start + num_features_this_dilation

#                 num_channels_this_combination = num_channels_per_combination[combination_index]

#                 num_channels_end = num_channels_start + num_channels_this_combination

#                 channels_this_combination = channel_indices[num_channels_start:num_channels_end]

#                 _padding1 = (_padding0 + kernel_index) % 2

#                 index_0, index_1, index_2 = indices[kernel_index]

#                 C = C_alpha[channels_this_combination] + \
#                     C_gamma[index_0][channels_this_combination] + \
#                     C_gamma[index_1][channels_this_combination] + \
#                     C_gamma[index_2][channels_this_combination]
#                 C = np.sum(C, axis = 0)

#                 if _padding1 == 0:
#                     for feature_count in range(num_features_this_dilation):
#                         features[example_index, feature_index_start + feature_count] = _PPV(C, biases[feature_index_start + feature_count]).mean()
#                 else:
#                     for feature_count in range(num_features_this_dilation):
#                         features[example_index, feature_index_start + feature_count] = _PPV(C[padding:-padding], biases[feature_index_start + feature_count]).mean()

#                 feature_index_start = feature_index_end

#                 combination_index += 1
#                 num_channels_start = num_channels_end

#     return features

In [ ]:
#export
from sktime.transformations.panel.rocket._minirocket import _fit as minirocket_fit
from sktime.transformations.panel.rocket._minirocket import _transform as minirocket_transform
from sktime.transformations.panel.rocket._minirocket_multivariate import _fit_multi as minirocket_fit_multi
from sktime.transformations.panel.rocket._minirocket_multivariate import _transform_multi as minirocket_transform_multi
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sklearn.linear_model import RidgeCV, RidgeClassifierCV

In [ ]:
#export
# This is a wrapper used to extract features in the format required by the tsai library. 

class MiniRocketFeatures:
    def __init__(self, standardize=False, by_sample=False, by_var=False, by_step=False):
        self.standardize = standardize or by_sample or by_var or by_step
        drop_axes = []
        if by_sample: drop_axes.append(0)
        if by_var: drop_axes.append(1)
        if by_step: drop_axes.append(2)
        self.axes = tuple([ax for ax in (0, 1, 2) if ax not in drop_axes])
        self.by_sample = by_sample
        
    def fit(self, o, num_features=10_000, max_dilations_per_kernel=32):
        if o.dtype != 'float32': o = o.astype('float32')
        if o.ndim == 2: 
            o = o[:, np.newaxis]
        if self.standardize:
            mean = o.mean(axis=self.axes, keepdims=True)
            std = o.std(axis=self.axes, keepdims=True) + 1e-8
            if not self.by_sample:
                self.mean, self.std = mean, std
        if o.shape[1] == 1:
            parameters = minirocket_fit(o[0, 0][np.newaxis], num_features=num_features, max_dilations_per_kernel=max_dilations_per_kernel)
        else: 
            parameters = minirocket_fit_multi(o[0][np.newaxis], num_features=num_features, max_dilations_per_kernel=max_dilations_per_kernel)
        self.parameters = parameters
    
    def transform(self, o, fname='X_tfm', path='./data/MiniRocketFeatures', on_disk=True, mode='c', chunksize=10_000):
        if o.dtype != 'float32': o = o.astype('float32')
        if o.ndim == 2: 
            o = o[:, np.newaxis]
        if self.standardize:
            if self.by_sample:
                mean = o.mean(axis=self.axes, keepdims=True)
                std = o.std(axis=self.axes, keepdims=True) + 1e-8
            else: 
                mean, std = self.mean, self.std
            o = (o - mean) / std
        if chunksize is None:
            if o.shape[1] == 1:
                o_tfm = minirocket_transform(o[:, 0], self.parameters)[..., np.newaxis]
            else:
                o_tfm = minirocket_transform_multi(o, self.parameters)[..., np.newaxis]
            return o_tfm
        else: 
            start = 0
            pb = progress_bar(range(math.ceil(len(o) / chunksize)), leave=False, comment="creating MiniRocket features")
            for i in pb:
                end = start + chunksize
                if o.shape[1] == 1:
                    _o_tfm = minirocket_transform(o[start:end, 0], self.parameters)[..., np.newaxis]
                else:
                    _o_tfm = minirocket_transform_multi(o[start:end], self.parameters)[..., np.newaxis]
                if i == 0: 
                    shape = (o.shape[0], _o_tfm.shape[1], _o_tfm.shape[2])
                    o_tfm = create_empty_array(shape, fname=fname, path=path, on_disk=on_disk, mode=mode)
                o_tfm[start:end] = _o_tfm
                start = end
                del _o_tfm
                gc.collect()
            return o_tfm
    
    def fit_transform(self, o, num_features=10_000, max_dilations_per_kernel=32, 
                        fname='X_tfm', path='./data/MiniRocketFeatures', on_disk=True, mode='c', chunksize=10_000):
        self.fit(o, num_features=num_features, max_dilations_per_kernel=max_dilations_per_kernel)
        return self.transform(o, fname=fname, path=path, on_disk=on_disk, mode=mode, chunksize=chunksize)
        
    def save(self, fname, path='./models/MiniRocketFeatures'):
        path = Path(path)
        if not fname.endswith('pkl'): fname = f'{fname}.pkl'
        filename = path/fname
        filename.parent.mkdir(parents=True, exist_ok=True)
        with open(filename, 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)

    def load(self, fname, path='./models/MiniRocketFeatures'):
        path = Path(path)
        if not fname.endswith('pkl'): fname = f'{fname}.pkl'
        filename = path/fname
        filename.parent.mkdir(parents=True, exist_ok=True)
        with open(filename, 'rb') as input:
            output = pickle.load(input)
        return output

In [ ]:
# export
# This is an unofficial MINIROCKET implementation in Pytorch developed by Ignacio Oguiza - timeseriesAI@gmail.com based on:
# Dempster, A., Schmidt, D. F., & Webb, G. I. (2020). MINIROCKET: A Very Fast (Almost) Deterministic Transform for Time Series Classification.
# arXiv preprint arXiv:2012.08791.
# Official repo: https://github.com/angus924/minirocket

class MiniRocket(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=1, fc_dropout=0., **kwargs):
        """
        MINIROCKET implementation where features are previously calculated.
        Args:
            c_in: number of features per sample. For 10_000 kernels iw will be 9996. 
            c_out: number of classes.
            seq_len: For MINIROCKET this is always 1 as features are previously calculated. Included for compatibility.
            fc_dropout: indicates whether dropout should be added to the last fully connected layer

        Input shape: [batch_size x c_in x 1]
        """
        backbone = nn.Sequential()
        self.head_nf = c_in
        layers = [Squeeze()]
        if fc_dropout:
            layers += [nn.Dropout(fc_dropout)]
        linear = nn.Linear(c_in, c_out)
        nn.init.constant_(linear.weight.data, 0)
        nn.init.constant_(linear.bias.data, 0)
        layers += [linear]
        head = nn.Sequential(*layers)
        super().__init__(OrderedDict([('backbone', backbone), ('head', head)]))

In [ ]:
dsid = 'ECGFiveDays'
X, y, splits = get_UCR_data(dsid, split_data=False)
mrf = MiniRocketFeatures()
mrf.fit(X[splits[0]])
X_tfm_train = mrf.transform(X[splits[0]], 'mrf_train', on_disk=True)
p1 = mrf.parameters
mrf.save(f'mrf_{dsid}')
del mrf
mrf = MiniRocketFeatures().load(f'mrf_{dsid}')
p2 = mrf.parameters
test_eq(p1, p2)
X_tfm_valid = mrf.transform(X[splits[1]], 'mrf_valid', on_disk=True)

In [ ]:
# #hide
# # This is an unofficial MINIROCKET implementation in Pytorch developed by Ignacio Oguiza - timeseriesAI@gmail.com based on:
# # Dempster, A., Schmidt, D. F., & Webb, G. I. (2020). MINIROCKET: A Very Fast (Almost) Deterministic Transform for Time Series Classification. 
# # arXiv preprint arXiv:2012.08791.
# # Official repo: https://github.com/angus924/minirocket

# class MINIROCKETv2(nn.Module):
#     def __init__(self, c_in, c_out, seq_len=None, num_features=10_000, max_dilations_per_kernel=32, fc_dropout=0., 
#                  eps=1e-8, custom_head=None, **kwargs):
#         super().__init__()
#         self.c_in, self.num_features, self.max_dilations_per_kernel, self.eps = c_in, num_features, max_dilations_per_kernel, eps
#         self.rocket_params = None
#         self.f_mean, self.f_std = None, None
#         self.fit = minirocket_fit if c_in == 1 else minirocket_fit_multi
#         self.tfm = minirocket_transform if c_in == 1 else minirocket_transform_multi
#         self.head_nf = 84 * (num_features // 84)
#         if custom_head is not None: 
#             self.head = custom_head(self.head_nf, c_out, 1, fc_dropout=fc_dropout, **kwargs)
#         else:
#             layers = []
#             if fc_dropout: layers += [nn.Dropout(fc_dropout)]
#             layers += [Squeeze(-1)]
#             linear = nn.Linear(self.head_nf, c_out)
#             nn.init.constant_(linear.weight.data, 0)
#             nn.init.constant_(linear.bias.data, 0)
#             layers += [linear]
#             self.head = nn.Sequential(*layers)
        
#     def forward(self, x):
#         with torch.no_grad():
#             if self.rocket_params is None:
#                 if self.c_in == 1: 
#                     self.rocket_params = self.fit(x.cpu().numpy()[:, 0], self.num_features, self.max_dilations_per_kernel)
#                 else: 
#                     self.rocket_params = self.fit(x.cpu().numpy(), self.num_features, self.max_dilations_per_kernel)
#             if self.c_in == 1: 
#                 x_tfm = x.new(self.tfm(x.cpu().numpy()[:, 0], self.rocket_params)).unsqueeze(-1)
#             else: 
#                 x_tfm = x.new(self.tfm(x.cpu().numpy(), self.rocket_params)).unsqueeze(-1)
#             if self.f_mean is None:
#                 self.f_mean = x_tfm.mean(0)
#                 self.f_std = x_tfm.std(0) + self.eps
#             x_tfm = (x_tfm - self.f_mean) / self.f_std
#         return self.head(x_tfm)

With MiniRocket we use need to:
1. Transform X --> X_tfm
2. Train a MiniRocket using previously calculated features (X_tfm)

In [ ]:
from tsai.data.all import *
from tsai.learner import *
dsid = 'ECGFiveDays'
X, y, splits = get_UCR_data(dsid, split_data=False)
tfms = [None, TSClassification()]
batch_tfms = TSStandardize(by_var=True)
mrf = MiniRocketFeatures()
X_tfm = mrf.fit_transform(X)
dls = get_ts_dls(X_tfm, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=256)
learn = ts_learner(dls, MiniRocket, metrics=accuracy)
learn.fit(10, 1e-4, cbs=ReduceLROnPlateau(factor=0.5, min_lr=1e-8, patience=10))

epoch,train_loss,valid_loss,accuracy,time
0,0.693147,0.526332,0.754936,00:00
1,0.582386,0.435147,0.796748,00:00
2,0.508791,0.368742,0.828107,00:00
3,0.453322,0.312181,0.861789,00:00
4,0.407588,0.262873,0.893148,00:00
5,0.368118,0.220747,0.914053,00:00
6,0.333500,0.185743,0.944251,00:00
7,0.303107,0.157302,0.959350,00:00
8,0.276519,0.134482,0.973287,00:00
9,0.253316,0.116239,0.979094,00:00


In [ ]:
# export
class MiniRocketClassifier(sklearn.pipeline.Pipeline):
    def __init__(self, num_features=10_000, max_dilations_per_kernel=32, random_state=None,
                 alphas=np.logspace(-3, 3, 7), normalize_features=True, memory=None, verbose=False, scoring=None, class_weight=None, **kwargs):
        """
        MiniRocketClassifier is recommended for up to 10k time series. 
        For a larger dataset, you can use MINIROCKET (in Pytorch).
        scoring = None --> defaults to accuracy.
        """
        self.steps = [('minirocketmultivariate', MiniRocketMultivariate(num_features=num_features, 
                                                                        max_dilations_per_kernel=max_dilations_per_kernel,
                                                                        random_state=random_state)),
                      ('ridgeclassifiercv', RidgeClassifierCV(alphas=alphas, 
                                                              normalize=normalize_features, 
                                                              scoring=scoring, 
                                                              class_weight=class_weight, 
                                                              **kwargs))]
        store_attr()
        self._validate_steps()

    def __repr__(self):
        return f'Pipeline(steps={self.steps.copy()})'

    def save(self, fname=None, path='./models'):
        fname = ifnone(fname, 'MiniRocketClassifier')
        path = Path(path)
        filename = path/fname
        with open(f'{filename}.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)


def load_minirocket(fname, path='./models'):
    path = Path(path)
    filename = path/fname
    with open(f'{filename}.pkl', 'rb') as input:
        output = pickle.load(input)
    return output

In [ ]:
# export
class MiniRocketRegressor(sklearn.pipeline.Pipeline):
    def __init__(self, num_features=10000, max_dilations_per_kernel=32, random_state=None,
                 alphas=np.logspace(-3, 3, 7), *, normalize_features=True, memory=None, verbose=False, scoring=None, **kwargs):
        """
        MiniRocketRegressor is recommended for up to 10k time series. 
        For a larger dataset, you can use MINIROCKET (in Pytorch).
        scoring = None --> defaults to r2.
        """
        self.steps = [('minirocketmultivariate', MiniRocketMultivariate(num_features=num_features,
                                                                        max_dilations_per_kernel=max_dilations_per_kernel,
                                                                        random_state=random_state)),
                      ('ridgecv', RidgeCV(alphas=alphas, normalize=normalize_features, scoring=scoring, **kwargs))]
        store_attr()
        self._validate_steps()

    def __repr__(self):
        return f'Pipeline(steps={self.steps.copy()})'

    def save(self, fname=None, path='./models'):
        fname = ifnone(fname, 'MiniRocketRegressor')
        path = Path(path)
        filename = path/fname
        with open(f'{filename}.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)


def load_minirocket(fname, path='./models'):
    path = Path(path)
    filename = path/fname
    with open(f'{filename}.pkl', 'rb') as input:
        output = pickle.load(input)
    return output

In [ ]:
#export
from sklearn.ensemble import VotingClassifier
class MiniRocketVotingClassifier(VotingClassifier):
    def __init__(self, n_estimators=5, weights=None, n_jobs=-1, num_features=10_000, max_dilations_per_kernel=32, random_state=None, 
                 alphas=np.logspace(-3, 3, 7), normalize_features=True, memory=None, verbose=False, scoring=None, class_weight=None, **kwargs):
        store_attr()
        estimators = [(f'clf{i}', MiniRocketClassifier(num_features=num_features, max_dilations_per_kernel=max_dilations_per_kernel, 
                                                       random_state=random_state, alphas=alphas, normalize_features=normalize_features, memory=memory, 
                                                       verbose=verbose, scoring=scoring, class_weight=class_weight, **kwargs)) 
                    for i in range(n_estimators)]
        super().__init__(estimators, voting='hard', weights=weights, n_jobs=n_jobs, verbose=verbose)

    def __repr__(self):   
        return f'MiniRocketVotingClassifier(n_estimators={self.n_estimators}, \nsteps={self.estimators[0][1].steps})'

    def save(self, fname=None, path='./models'):
        fname = ifnone(fname='MiniRocketVotingClassifier')
        path = Path(path)
        filename = path/fname
        filename.parent.mkdir(parents=True, exist_ok=True)
        with open(f'{filename}.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)

            
def get_minirocket_preds(X, fname, path='./models', model=None):
    if X.ndim == 1: X = X[np.newaxis][np.newaxis]
    elif X.ndim == 2: X = X[np.newaxis]
    if model is None: 
        model = load_minirocket(fname=fname, path=path)
    return model.predict(X)

In [ ]:
# export
from sklearn.ensemble import VotingRegressor

class MiniRocketVotingRegressor(VotingRegressor):
    def __init__(self, n_estimators=5, weights=None, n_jobs=-1, num_features=10_000, max_dilations_per_kernel=32, random_state=None,
                 alphas=np.logspace(-3, 3, 7), normalize_features=True, memory=None, verbose=False, scoring=None, **kwargs):
        store_attr()
        estimators = [(f'clf{i}', MiniRocketRegressor(num_features=num_features, max_dilations_per_kernel=max_dilations_per_kernel,
                                                      random_state=random_state, alphas=alphas, normalize_features=normalize_features, memory=memory,
                                                      verbose=verbose, scoring=scoring, **kwargs))
                      for i in range(n_estimators)]
        super().__init__(estimators, weights=weights, n_jobs=n_jobs, verbose=verbose)

    def __repr__(self):
        return f'MiniRocketVotingRegressor(n_estimators={self.n_estimators}, \nsteps={self.estimators[0][1].steps})'

    def save(self, fname=None, path='./models'):
        fname = ifnone(fname='MiniRocketVotingRegressor')
        path = Path(path)
        filename = path/fname
        filename.parent.mkdir(parents=True, exist_ok=True)
        with open(f'{filename}.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
# #hide
# from tsai.data.all import *
# from tsai.learner import *
# tfms = [None, TSClassification()]
# batch_tfms = [TSMagScale(), TSStandardize(by_sample=True)]
# dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=256)
# learn = ts_learner(dls, MINIROCKETv2, metrics=accuracy)
# learn.fit(1, 1e-4, cbs=ReduceLROnPlateau(factor=0.5, min_lr=1e-8, patience=10))

In [ ]:
# #hide
# from sklearn.pipeline import make_pipeline
# from sktime.transformations.panel.rocket import MiniRocketMultivariate
# from sklearn.linear_model import LogisticRegressionCV
# from sklearn.preprocessing import StandardScaler

# class MiniRocketLR(sklearn.pipeline.Pipeline):
#     def __init__(self, num_features=10_000, max_dilations_per_kernel=32, random_state=None, memory=None, verbose=False, **kwargs):
#         self.steps = [('minirocketmultivariate', MiniRocketMultivariate(num_features=num_features, 
#                                                                         max_dilations_per_kernel=max_dilations_per_kernel, 
#                                                                         random_state=random_state)),
#                       ('standardscaler', StandardScaler()),
#                       ('logisticregressioncv', LogisticRegressionCV(**kwargs))]
#         self.num_features, self.max_dilations_per_kernel, self.random_state = num_features, max_dilations_per_kernel, random_state
#         self.kwargs = kwargs
#         self.memory = memory
#         self.verbose = verbose
#         self._validate_steps()

In [ ]:
# Univariate classification with sklearn-type API
dsid = 'OliveOil'
fname = 'MiniRocketClassifier'
X_train, y_train, X_test, y_test = get_UCR_data(dsid)
cls = MiniRocketClassifier()
cls.fit(X_train, y_train)
cls.save(fname)
pred = cls.score(X_test, y_test)
del cls
cls = load_minirocket(fname)
test_eq(cls.score(X_test, y_test), pred)

In [ ]:
# Multivariate classification with sklearn-type API
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = get_UCR_data(dsid)
cls = MiniRocketClassifier()
cls.fit(X_train, y_train)
cls.score(X_test, y_test)

0.9

In [ ]:
# Multivariate classification with sklearn-type API
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = get_UCR_data(dsid)
cls = MiniRocketVotingClassifier(5)
cls.fit(X_train, y_train)
cls.score(X_test, y_test)

0.9111111111111111

In [ ]:
# Univariate regression with sklearn-type API
from sklearn.metrics import mean_squared_error
dsid = 'Covid3Month'
fname = 'MiniRocketRegressor'
X_train, y_train, X_test, y_test = get_Monash_data(dsid)
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
reg = MiniRocketRegressor(scoring=rmse_scorer)
reg.fit(X_train, y_train)
reg.save(fname)
del reg
reg = load_minirocket(fname)
y_pred = reg.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

0.041647950237517425

In [ ]:
# Multivariate regression with sklearn-type API
from sklearn.metrics import mean_squared_error
dsid = 'AppliancesEnergy'
X_train, y_train, X_test, y_test = get_Monash_data(dsid)
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
reg = MiniRocketRegressor(scoring=rmse_scorer)
reg.fit(X_train, y_train)
reg.save(fname)
del reg
reg = load_minirocket(fname)
y_pred = reg.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

2.2571082932302287

In [ ]:
# Multivariate regression ensemble with sklearn-type API
reg = MiniRocketVotingRegressor(5, scoring=rmse_scorer)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

2.2923262761032364

In [ ]:
#hide
out = create_scripts(); beep(out)

<IPython.core.display.Javascript object>

Converted 000_utils.ipynb.
Converted 000b_data.validation.ipynb.
Converted 000c_data.preparation.ipynb.
Converted 001_data.external.ipynb.
Converted 002_data.core.ipynb.
Converted 003_data.preprocessing.ipynb.
Converted 003b_data.transforms.ipynb.
Converted 003c_data.mixed_augmentation.ipynb.
Converted 003d_data.image.ipynb.
Converted 003e_data.features.ipynb.
Converted 005_data.tabular.ipynb.
Converted 006_data.mixed.ipynb.
Converted 007_metrics.ipynb.
Converted 008_learner.ipynb.
Converted 009_optimizer.ipynb.
Converted 010_callback.core.ipynb.
Converted 011_callback.noisy_student.ipynb.
Converted 012_callback.gblend.ipynb.
Converted 013_callback.TSBERT.ipynb.
Converted 014_callback.PredictionDynamics.ipynb.
Converted 100_models.layers.ipynb.
Converted 100b_models.utils.ipynb.
Converted 100c_models.explainability.ipynb.
Converted 101_models.ResNet.ipynb.
Converted 101b_models.ResNetPlus.ipynb.
Converted 102_models.InceptionTime.ipynb.
Converted 102b_models.InceptionTimePlus.ipynb.
Co